I have downloaded the latest influxd.exe and unzipped it to ProgramData\InfluxData
I have also downloaded the latest influx.exe (influxdb client) and unzipped it to ProgramData\InfluxData

influxdb and influxdb_client are installed with pip for this python environment

open powershell and run influx.exe setup
the influx web ui will be at localhost:8086
also run influxd.exe and leave the terminal running

In [1]:
from influxdb_client import InfluxDBClient
from influxdb_client.client.write_api import SYNCHRONOUS
from datetime import datetime

In [2]:
#create connection to database
client = InfluxDBClient(url="http://localhost:8086",token="Cxm9NsNCekh3eC1OWCQQ3ENhPUnU4u1ODq449VplcwRUoS7IDzTHtTKX_3Bl1DnvVPxQnEhTeCLAR3ab5msmEg==",org="Ge-data-project")


In [3]:
#create write_api and query_api instances
bucket = "GEItemPrices"
#synchronous handles errors better than asynchronous
write_api = client.write_api(write_options=SYNCHRONOUS)
query_api = client.query_api()

#one way to write data to bucket
#Measurement name + Tag set + Field set + timestamp

#create records where one measurement is the data for one item.
#for one 5m data returned from the api call, make a record for each and append to records
records = [
    {
        "measurement": "itemPrice",
        "tags": {"itemID": str('itemID')},
        "fields": {"avgHighPrice": 8907,
                   "avgLowPrice": 7890,
                   "highPriceVolume": 5675756,
                   "lowPriceVolume": 5675744},
        "time": datetime.now()
    },
    {
        "measurement": "itemPrice",
        "tags": {"itemID": str('diffitemID')},
        "fields": {"avgHighPrice": 8907,
                   "avgLowPrice": 7890,
                   "highPriceVolume": 5675756,
                   "lowPriceVolume": 5675744},
        "time": datetime.now()
    }
]
#fields are avgHighPrice and avgLowPrice; lowPriceVolume and highPriceVolume
#write records to the bucket
write_api.write(bucket=bucket, record=records)


In [4]:
#request data in dataFrame format
#specify columns to keep in the response

#range if for the time to get data from. Can specify start and end times
#use a filter function to specify what itemIDs you want to get data for
#pivot and keep columns to specify what data you want in the response
data_frame = query_api.query_data_frame('from(bucket:"GEItemPrices") '
                                        '|> range(start: -10h) '
                                        '|> filter(fn: (r) => r.itemID == "itemID")'
                                        '|> pivot(rowKey:["_time"], columnKey: ["_field"], valueColumn: "_value") '
                                        '|> keep(columns: ["avgHighPrice", "avgLowPrice","highPriceVolume","lowPriceVolume","_time","itemID"])')
# print(data_frame.to_string())


In [5]:
data_frame

,result,table,_time,itemID,avgHighPrice,avgLowPrice,highPriceVolume,lowPriceVolume
0,_result,0,2023-03-04 17:02:00.545490+00:00,itemID,8907,7890,5675756,5675744
1,_result,0,2023-03-04 17:02:37.305739+00:00,itemID,8907,7890,5675756,5675744
2,_result,0,2023-03-04 17:11:36.040441+00:00,itemID,8907,7890,5675756,5675744
3,_result,0,2023-03-04 20:36:05.474287+00:00,itemID,8907,7890,5675756,5675744
4,_result,0,2023-03-04 20:41:40.539336+00:00,itemID,8907,7890,5675756,5675744
5,_result,0,2023-03-04 20:52:39.906475+00:00,itemID,8907,7890,5675756,5675744
6,_result,0,2023-03-04 21:14:54.575921+00:00,itemID,8907,7890,5675756,5675744
7,_result,0,2023-03-04 21:15:22.919332+00:00,itemID,8907,7890,5675756,5675744
8,_result,0,2023-03-04 21:15:23.421078+00:00,itemID,8907,7890,5675756,5675744
9,_result,0,2023-03-04 21:15:23.822331+00:00,itemID,8907,7890,5675756,5675744


In [6]:
client.close()